In [1]:
# !pip install s3fs
# !pip install ../dist/aidial_rag_eval-0.1.0-py3-none-any.whl

In [2]:
import json
import pandas as pd
from aidial_rag_eval.evaluate import evaluate

In [3]:
ground_truth = "s3://deltix-staging-dial-rag-eval/ground_truth/epam10k/ground_truth_v2.parquet"
answers = "s3://deltix-staging-dial-rag-eval/answers/epam10k/answers_azure_rag.parquet"
metrics = "s3://deltix-staging-dial-rag-eval/evaluation/tmp/metrics_tmp.parquet"

In [4]:
pd.read_parquet(ground_truth).head()

,question,answer,facts,documents
0,Full legal name of the Company,"EPAM SYSTEMS, INC.","[EPAM SYSTEMS, INC.\n\n(Exact name of registra...",[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
1,Address,"42 University Drive, Suite 202, Newtown, Penns...","[41 University Drive, Suite 202, Newtown, Penn...",[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
2,Provide your corporate HQ location and geograp...,Corporate HQ location is in 41 University Driv...,"[EPAM SYSTEMS, INC.\n\n(Exact name of registra...",[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
3,How many staff members/resources do you have a...,India is one of EPAM largest delivery location...,"[Outside of Ukraine and Belarus, our largest d...",[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
4,"Main Office Location (City, State and Country)",Our principal executive offices are located at...,"[EPAM Systems, Inc. was incorporated in the St...",[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...


In [5]:
pd.read_parquet(answers).head()

,question,answer,context,documents
0,Full legal name of the Company,The full legal name of the company is EPAM Sys...,[Chief\nAccounting Officer \n\n(principal acco...,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
1,Address,The requested information is not available in ...,[],[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
2,Provide your corporate HQ location and geograp...,"EPAM Systems, Inc.'s corporate headquarters ar...",[may not be certain.\nOur determination of tax...,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
3,How many staff members/resources do you have a...,The requested information is not available in ...,"[had approximately 59,300, 58,800, and 41,150 ...",[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...
4,"Main Office Location (City, State and Country)","The main office of EPAM Systems, Inc. is locat...",[or systems we develop for them. Our suppliers...,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...


In [6]:
metrics_dataset = evaluate(ground_truth, answers, metrics)
print(metrics_dataset.data_path)
print(metrics_dataset.metadata.metrics)

s3://deltix-staging-dial-rag-eval/evaluation/tmp/metrics_tmp.parquet
{'recall': 0.6617647058823529, 'precision': 0.24789915966386555, 'mrr': 0.44305555555555554, 'f1': 0.32180629827688656}


In [7]:
metrics_df = metrics_dataset.read_dataframe()
metrics_df.head()

,documents,question,facts,context,facts_ranks,context_relevance,context_highlight,recall,precision,mrr,f1
0,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...,Full legal name of the Company,"[EPAM SYSTEMS, INC.\n\n(Exact name of registra...",[Chief\nAccounting Officer \n\n(principal acco...,[-1],"[0, 0]","[{""match"": [{""text"": ""Chief\nAccounting Office...",0.0,0.000000,0.000000,0.00
1,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...,Address,"[41 University Drive, Suite 202, Newtown, Penn...",[],[-1],[],[],0.0,0.000000,0.000000,0.00
2,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...,Provide your corporate HQ location and geograp...,"[EPAM SYSTEMS, INC.\n\n(Exact name of registra...",[may not be certain.\nOur determination of tax...,"[-1, -1]","[0, 0, 0, 0, 0, 0, 0]","[{""match"": [{""text"": ""may not be certain.\nOur...",0.0,0.000000,0.000000,0.00
3,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...,How many staff members/resources do you have a...,"[Outside of Ukraine and Belarus, our largest d...","[had approximately 59,300, 58,800, and 41,150 ...",[2],"[0, 0, 1, 0]","[{""match"": [{""text"": ""had approximately 59,300...",1.0,0.250000,0.333333,0.40
4,[https://d18rn0p25nwr6d.cloudfront.net/CIK-000...,"Main Office Location (City, State and Country)","[EPAM Systems, Inc. was incorporated in the St...",[or systems we develop for them. Our suppliers...,[0],"[1, 0, 0, 0, 0, 0, 0]","[{""match"": [{""text"": ""or systems we develop fo...",1.0,0.142857,1.000000,0.25


In [8]:
metrics_df[["facts", "context", "facts_ranks", "context_relevance"]].head()

,facts,context,facts_ranks,context_relevance
0,"[EPAM SYSTEMS, INC.\n\n(Exact name of registra...",[Chief\nAccounting Officer \n\n(principal acco...,[-1],"[0, 0]"
1,"[41 University Drive, Suite 202, Newtown, Penn...",[],[-1],[]
2,"[EPAM SYSTEMS, INC.\n\n(Exact name of registra...",[may not be certain.\nOur determination of tax...,"[-1, -1]","[0, 0, 0, 0, 0, 0, 0]"
3,"[Outside of Ukraine and Belarus, our largest d...","[had approximately 59,300, 58,800, and 41,150 ...",[2],"[0, 0, 1, 0]"
4,"[EPAM Systems, Inc. was incorporated in the St...",[or systems we develop for them. Our suppliers...,[0],"[1, 0, 0, 0, 0, 0, 0]"


In [9]:
metrics_df[["facts_ranks", "context_relevance", "recall", "precision", "mrr", "f1"]].head()

,facts_ranks,context_relevance,recall,precision,mrr,f1
0,[-1],"[0, 0]",0.0,0.000000,0.000000,0.00
1,[-1],[],0.0,0.000000,0.000000,0.00
2,"[-1, -1]","[0, 0, 0, 0, 0, 0, 0]",0.0,0.000000,0.000000,0.00
3,[2],"[0, 0, 1, 0]",1.0,0.250000,0.333333,0.40
4,[0],"[1, 0, 0, 0, 0, 0, 0]",1.0,0.142857,1.000000,0.25


In [10]:
print(metrics_df["context"].iloc[3][2])

2022, Belarus had approximately 4,500
delivery professionals. During 2022, a significant number of our employees in Russia and Belarus have relocated, thus increasing the size of our other delivery locations.
Outside of Ukraine and Belarus, our largest delivery locations are India and Poland with approximately 5,900 and 5,650 delivery professionals, respectively, as of December 31,
2022.

Human Capital

Our employees are a key factor in our ability to grow our revenues and serve our customers. We believe the quality of our employees serves as a key point of
differentiation in how we deliver a superior value proposition to our customers and investors. Therefore, it is critical to our success that we are able to identify, attract, hire and
retain delivery professionals who are highly skilled in information technology to execute our services, as well as individuals with appropriate skills to fill our executive, finance,
legal, human resources and other key management positions. To attract

In [11]:
print(json.dumps(json.loads(metrics_df["context_highlight"].iloc[3][2]), indent=2))

{
  "match": [
    {
      "text": "2022, Belarus had approximately 4,500\ndelivery professionals. During 2022, a significant number of our employees in Russia and Belarus have relocated, thus increasing the size of our other delivery locations.\n",
      "facts": []
    },
    {
      "text": "Outside of Ukraine and Belarus, our largest delivery locations are India and Poland with approximately 5,900 and 5,650 delivery professionals, respectively, as of December 31,\n2022.\n\n",
      "facts": [
        0
      ]
    },
    {
      "text": "Human Capital\n\nOur employees are a key factor in our ability to grow our revenues and serve our customers. We believe the quality of our employees serves as a key point of\ndifferentiation in how we deliver a superior value proposition to our customers and investors. Therefore, it is critical to our success that we are able to identify, attract, hire and\nretain delivery professionals who are highly skilled in information technology to execute ou